In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16

train_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\training'
val_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\face\LyingLegRaise\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 15894 images belonging to 16 classes.
Found 3415 images belonging to 16 classes.
Found 3406 images belonging to 16 classes.


In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# VGG16의 모든 층을 동결
for layer in base_model.layers:
    if layer.name == 'block5_conv1' or layer.name == 'block5_conv2':
        layer.trainable = True
    else:
       layer.trainable = False
    
base_model.summary()
 
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dropout(0.1))  # 과적합 방지를 위한 드롭아웃 추가
model.add(layers.Dense(16, activation='softmax'))  

model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00011),
              metrics=['acc'])

# 모델 훈련
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,    
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25,
    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)


Epoch 1/25
496/496 [==============================] - 25s 43ms/step - loss: 2.2490 - acc: 0.2126 - val_loss: 1.9945 - val_acc: 0.2854
Epoch 2/25
496/496 [==============================] - 21s 41ms/step - loss: 1.8469 - acc: 0.3326 - val_loss: 1.8511 - val_acc: 0.3343
Epoch 3/25
496/496 [==============================] - 20s 40ms/step - loss: 1.5506 - acc: 0.4366 - val_loss: 1.5188 - val_acc: 0.4531
Epoch 4/25
496/496 [==============================] - 21s 42ms/step - loss: 1.2905 - acc: 0.5287 - val_loss: 1.4682 - val_acc: 0.4746
Epoch 5/25
496/496 [==============================] - 19s 39ms/step - loss: 1.0813 - acc: 0.6099 - val_loss: 1.2470 - val_acc: 0.5663
Epoch 6/25
496/496 [==============================] - 19s 39ms/step - loss: 0.8978 - acc: 0.6764 - val_loss: 1.1383 - val_acc: 0.6082
Epoch 7/25
496/496 [==============================] - 19s 39ms/step - loss: 0.7453 - acc: 0.7372 - val_loss: 1.0561 - val_acc: 0.6330
Epoch 8/25
496/496 [==============================] - 19s 39ms